# INTRODUCTION TENSORBOARD

## ARCHITECTURE 

![AddingConstants](images/softmax-image)

### STEP 1: IMPORT LIBRARY AND LOAD DATASET

In [1]:
import tensorflow as tf
import numpy as np
import os

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

learning_rate = 0.003
num_iterations = 10000
LOGDIR = './logs/4-mnist-basic-output/'

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### STEP 2: CREATE AN INTERACTIVE SESSION

In [2]:
sess = tf.InteractiveSession()

### STEP 3: DEFINE SHAPE OF OUR DATASET

In [3]:
X = tf.placeholder(tf.float32, shape=[None, 784])
Y_ = tf.placeholder(tf.float32, shape=[None, 10])

### STEP 4: DEFINE HELPER FUNCTIONS

### STEP 5: DEFINE MODEL

In [4]:

with tf.name_scope('softmax'):

    W = tf.Variable(tf.truncated_normal([784, 10], stddev=0.1), name='W')
    b = tf.Variable(tf.constant(0.1, shape=[10]), name='B')
    YLogits = tf.matmul(X, W) + b
    Y = tf.nn.softmax(YLogits)
    tf.summary.histogram('weights', W)
    tf.summary.histogram('biases', b)
    tf.summary.histogram('activations', Y)


### STEP 6: DEFINE METRICS

#### 1. LOSS IN THE FORM OF CROSS ENTROPY

In [5]:
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=YLogits, labels=Y_), name='xent')
    tf.summary.scalar('xent', cross_entropy)

#### 2. TRAINING USING GRADIENT DESCENT OPTIMIZER

In [6]:
with tf.name_scope("train"):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

#### 3. ACCURACY OF THE MODEL

In [7]:
with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("accuracy", accuracy)

### STEP 7: TRAIN MODEL

#### Merge all summary operations into one

In [8]:
summ = tf.summary.merge_all()

#### Create a saver object

In [9]:
saver = tf.train.Saver()

#### Initialize all the variables

In [10]:
sess.run(tf.global_variables_initializer())

#### Create FileWriters to write summaries to disk

In [11]:
writer_train = tf.summary.FileWriter(LOGDIR+'train/')
writer_test = tf.summary.FileWriter(LOGDIR+'test/')

#### Add the computational graphs we built to the writers

In [12]:
writer_train.add_graph(sess.graph)
writer_test.add_graph(sess.graph)

In [14]:
for i in range(num_iterations):
    X_batch, Y_batch = mnist.train.next_batch(100)
    X_batch.astype(np.float32)
    Y_batch.astype(np.float32)
    fd = {X:X_batch, Y_:Y_batch}
    [train_accuracy, s] = sess.run([accuracy, summ], feed_dict=fd)
    writer_train.add_summary(s, i)

    if i%50 == 0:
        [test_accuracy, cross_entropy_loss, s] = sess.run([accuracy, cross_entropy, summ], feed_dict={X:mnist.test.images,Y_:mnist.test.labels})
        writer_test.add_summary(s, i)
        saver.save(sess, os.path.join(LOGDIR, "model.ckpt"), i)
        
    if i%1000 == 0:
        print_str = '\n\n\nStep ' + str(i) + '\n\tTrain Accuracy: ' + str(train_accuracy) + '\n\tTest Accuracy: ' + str(test_accuracy) + '\n\tCross Entropy Loss: '+ str(cross_entropy_loss)
        print(print_str)
    sess.run([train_step], feed_dict={X: X_batch, Y_: Y_batch})




Step 0
	Train Accuracy: 0.94
	Test Accuracy: 0.9269
	Cross Entropy Loss: 0.272768



Step 1000
	Train Accuracy: 0.96
	Test Accuracy: 0.9255
	Cross Entropy Loss: 0.278662



Step 2000
	Train Accuracy: 0.96
	Test Accuracy: 0.9243
	Cross Entropy Loss: 0.280488


KeyboardInterrupt: 

### STEP 8: EVALUATE MODEL

In [ ]:
final_accuracy = accuracy.eval(feed_dict={X: mnist.test.images, Y_: mnist.test.labels})
print('FINAL ACCURACY:', final_accuracy)

### STEP 9: TEST PREDICTION

In [ ]:
x_actual = mnist.test.images[0:1]
y_actual = mnist.test.labels[0:1]

print('ACTUAL VALUE:', y_actual)

In [ ]:
y_predicted = sess.run(Y, feed_dict={X: x_actual})

print('PREDICTED VALUE:', y_predicted)

In [ ]:
output_actual = sess.run(tf.argmax(y_actual, 1))
output_predicted = sess.run(tf.argmax(y_predicted, 1))

print ('EXPECTED OUTPUT:', output_actual)
print ('PREDICTED OUTPUT:', output_predicted)

## ACCURACY GRAPH

![Title](images/4-basic-model-accuracy.png)

## LOSS GRAPH

![AddingConstants](images/4-basic-model-loss.png)

## COMPUTATIONAL GRAPH

![AddingConstants](images/4-basic-model-graph.png)